### work on hand-tagged American intercultural encounter segments

#### 1: Encode Am tags from ledger into filenames; only include expl and/or enc tagged

- name format:
    - vol_chap-date-nat-enc-expl-title-pagerange
    - enc_full enc_part enc_min enc_none
    - expl_y expl_n

underscores within features, dashes between features

- for entry in ledger:
    - if enc OR expl:
        - locate file by vol-chap
        - save file in new dir with new name

#### 2: prepare ledger for re-integration with rest of corpus

In [45]:
# in retrospect, the code below missed a bunch of files; not sure what happened
old_path = 'text-data/CC_ML_FR_trimmed_MAlem_AMER_cat/'
new_path = 'text-data/CC_ML_FR_trimmed_MAlem_AMER_cat_mod/'

import csv
import os
import re


with open('text-data/AMledger.csv', mode = 'r') as ledger_csv:
    ledger = csv.DictReader(ledger_csv)
    for row in ledger:
        #print('starting row')
        if 'v' in row['expl'] or 'v' in row['enc'] or 'v' in row['enc_min']:
            #print('row of interest')
            # if entry addresses travel or cultural encounter:
            filelist = os.scandir(old_path)
            for entry in filelist:
                #print('scanning' , entry.name)
                #print ( 'condition checks: ', row['vol'].zfill(2), re.search(r'_(.*?)_', entry.name).group(1), row['vol'].zfill(2) == entry.name[:2], row['chap'] == re.search(r'_(.*?)_', entry.name).group(1))
                if row['vol'].zfill(2) == entry.name[:2] and (row['chap'].zfill(2) == str(float(re.search(r'_(.*?)_', entry.name).group(1))) or row['chap'].zfill(2) == re.search(r'_(.*?)_', entry.name).group(1)):
                    #print(entry.name, 'passed')
                    # find the relevant text file
                    with open(entry.path, 'r', encoding="utf8") as f:
                        text = f.read()
                        expl = 'expl_y' if row['expl'] == 'v' else 'expl_n'
                        if row['enc'] == 'v' and row['expl'] != 'v':
                            enc = 'enc_full'
                        elif row['enc'] == 'v':
                            enc = 'enc_part'
                        elif row['enc_min'] == 'v':
                            enc = 'enc_min'
                        else:
                            enc = 'enc_none'
                        # determine name factors
                        name = row['vol'].zfill(2) + '_' + row['chap'].zfill(2) + '-' + row['date'] + '-' + row['nat'] + '-' + enc + '-' + expl + '-' + row['title'] + '-' + row['pages'].replace('-', '_') + '.txt'
                        # string formatted filename
                        with open((new_path + '/' + name), 'w', encoding="utf8") as fw:
                            fw.write(text)
                            #print('text outputted')
                    #break
                            
                            

In [24]:
import csv
import os
import re
import string
punctuation = [i for i in string.punctuation]

In [198]:
def get_vol(name):
    return(name[:2])
def get_chap(name):
    name = name[3:]
    return(name[:name.find('_')])
def wordcount(text):
    '''splits texts on whitespace and counts the segments, ignoring punctuation'''
    tokens = text.split()
    words = [w for w in tokens if w not in punctuation]
    return(len(words))
def wordcount_f(vol, chap):
    '''locates file, splits texts on whitespace and counts the segments, ignoring punctuation'''
    filelist = os.scandir('text-data/CC_ML_FR_trimmed_morphad_lem')
    for entry in filelist:
        if int(entry.name[:2]) == int(vol):
            if int(entry.name[3:5]) == int(chap):
                with open(entry.path, mode = 'r', encoding="utf8") as f:
                    text = f.read()
                    return(wordcount(text))

In [195]:
testseg = '07_17.2_AMER_1170_The_most_ancient_voyage_and_discovery_of_the_West_Indies_performed_by_Madoc_the_sonne_of_Owen_Guined_pp.133-135.txt'
testfull = '09_24_AMER_1572_The_memorable_voyage_of_M_John_Chilton_to_all_the_principall_parts_of_Nueva_Espanna_pp.360-377'
#print(get_chap(test))
print(testfull[3:5])

24


In [201]:
chap = '22.4'
print(chap[:chap.find('.')])

22


In [222]:
missing_files = ['09_17.2_AMER_1540_The_voyage_of_the_right_worshipfull_knight_Francisco_de_Ulloa_pp.217-267.txt']
missing_paths = ['text-data/CC_ML_FR_trimmed_MAlem_AMER_cat/' + f for f in missing_files]
for path in missing_paths:
    with open(path, mode = 'r', encoding="utf8") as f:
        text = f.read()
        print(wordcount(text))

18963


#### 2: prepare ledger for re-integration with rest of corpus

1. compute & record segment wordcount
1. [manually merge enc ledger with full ledger]
1. compute & record full chapter wordcounts
1. compute section fractions
1. update the following fields from sections into parent chapters: national_perspective, exploration_component, intercultural_encounter_component, minor_intercultural_encounter_component

In [217]:
cat_path = 'text-data/CC_ML_FR_trimmed_MAlem_AMER_cat/'
parent_path = 'text-data/CC_ML_FR_trimmed_morphad_lem'

with open('text-data/ledger_merged_1.csv', mode = 'r') as ledger_csv:
    full_ledger = csv.DictReader(ledger_csv) 
    with open('text-data/AMledger_wordcounts.csv', mode = 'r') as ledger2_csv:
        AM_ledger = csv.DictReader(ledger2_csv)
        #writing header row
        with open('text-data/ledger_merged_2.csv', mode = 'a', newline='') as ledger_m2_csv:
            writer = csv.DictWriter(ledger_m2_csv, ledger.fieldnames)
            writer.writer.writerow(ledger.fieldnames)
            #copy over all non-Am rows from full ledger + wordcounts + sect_f = 1 (includes header #2)
            for row in full_ledger:
                if row['geog'] != 'AMER':
                    #except header # 2 from wordcount/sect
                    if row['vol'] != 'volume':
                        row['word'] = wordcount_f(row['vol'], row['chap'])
                        row['sect_frac'] = 1
                    writer.writerow(row)
            #for Am rows:
            #for full chapters in am ledger, just copy am rows into the new ledgers  + sect_f = 1
            #when catches section: find corresponding full chapter;  + wordcount + sect_f = 1
            # then write in all the sectioned rows while calculating fractions
            #starting scan on am_ledger
            for row in AM_ledger:
                #for non-header rows:
                if row['geog'] == 'AMER':
                    if '.' not in row['chap']:
                        #for full chapters in am ledger, just copy am rows into the new ledgers +  sect_f = 1
                        filelist = os.scandir(parent_path)
                        row['sect_frac'] = 1
                        writer.writerow(row)
                    else:
                        #sequence of sections begins:  find corresponding full chapter row, write in + wordcount + sect_f = 1
                                                    # then write in all the sectioned rows while calculating fractions
                        #print(row)
                        if row['chap'][-1] == '1':
                            #bring over row from ful ledger
                            vol = row['vol']
                            chap = row['chap'][:row['chap'].find('.')]
                            print(f'looking for {vol}, {chap}')
                            with open('text-data/ledger_merged_1.csv', mode = 'r') as ledger3_csv:
                                full_ledger2 = csv.DictReader(ledger3_csv) 
                                for row_f in full_ledger2:
                                    #print(row_f)
                                    if row_f['vol'] != 'volume':
                                        if int(row_f['vol']) == int(vol) and int(row_f['chap']) == int(chap):
                                            wordcount_full = wordcount_f(vol, chap)
                                            row_f['word'] = wordcount_full
                                            row_f['sect_frac'] = 1
                                            writer.writerow(row_f)
                                            break
                        row['sect_frac'] = int(row['word']) / wordcount_full
                        writer.writerow(row)

looking for 7, 23
looking for 7, 30
looking for 8, 14
looking for 8, 49
looking for 9, 1
looking for 9, 2
looking for 9, 15
looking for 9, 17
looking for 10, 9
looking for 10, 23
looking for 11, 20
looking for 11, 25


['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
